## Model one trade variables

This notebook extracts the selected trade variables in the `indicator_list` from IMF and World Bank (wb) data sources, and writes them to a csv file.

In [11]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [12]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:20,.2f}'.format

|  variable                 | origin            | source      |granularity|countries|   description                                               | composition                                                       |
| --------------------------|-------------------|-------------|-----------|---------|-------------------------------------------------------------|-------------------------------------------------------------------|
| imports                   | -                 | wb econ     | yearly    | 217     | Imports of goods and services (% of GDP)                     | -                                                     |
| manufacturing value       | -                 | wb econ     | yearly    | 217     | Manufacturing, value added (% of GDP)                        | -                                                     |
| consumption per capita    | -                 | wb econ     | yearly    | 217     | Households and NPISHs Final consumption expenditure per capita (2010 USD)| -                                    |
| food exports              | -                 | wb econ     | yearly    | 217     | Food exports (% of merchandise exports)                    | -                                                     |
| energy imports            | -                 | wb econ     | yearly    | 217     | Energy imports, net (% of energy use)                        | -                                                     |
| consumption per capita    | -                 | wb econ     | yearly    | 217     | Households and NPISHs Final consumption expenditure per capita (constant 2010 USD)| -                           |
| GNI per capita            | -                 | wb econ     | yearly    | 217     | GNI per capita (constant 2010 USD)                        | -                                                     |
| food imports              | -                 | wb econ     | yearly    | 217     | Food imports (% of merchandise imports)                    | -                                                     |
| manufacturing value USD   | -                 | wb econ     | yearly    | 217     | Manufacturing, value added (constant 2010 USD)         | -                                                     |
| exports                   | -                 | wb econ     | yearly    | 217     | Exports of goods and services (% of GDP)                     | -                                    |
| trade share of GDP        | -                 | wb econ     | yearly    | 217     | Trade (% of GDP)                                             | -                                                     |
| high-tech exports         | -                 | wb econ     | yearly    | 217     | High-technology exports (% of manufactured exports)       | -                                                     |
| imports USD               | -                 | wb econ     | yearly    | 217     | Imports of goods and services (constant 2010 USD)         | -                                                     |
| exports USD               | -                 | wb econ     | yearly    | 217     | Exports of goods and services (constant 2010 USD)         | -                                                     |
| services trade            | -                 | wb econ     | yearly    | 217     | Trade in services (% of GDP)                                 | -                                                     |

In [58]:
indicator_list = ['Imports of goods and services (% of GDP)', 'Manufacturing, value added (% of GDP)',
                  'Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)',
                  'Food exports (% of merchandise exports)', 'Energy imports, net (% of energy use)',
                  'GNI per capita (constant 2010 US$)', 'Food imports (% of merchandise imports)',
                  'Manufacturing, value added (constant 2010 US$)', 'Exports of goods and services (% of GDP)',
                  'Trade (% of GDP)', 'High-technology exports (% of manufactured exports)',
                  'Imports of goods and services (constant 2010 US$)', 'Trade in services (% of GDP)',
                  'Exports of goods and services (constant 2010 US$)']

In [59]:
len(indicator_list)

14

## Load imf monthly data

In [60]:
%%bash
wc -l imf/*.csv

  365536 data/imf/BOP_11-25-2018 19-15-19-60_timeSeries.csv
      64 data/imf/COMMP_11-25-2018 19-13-52-15_timeSeries.csv
   14430 data/imf/CPI_11-25-2018 19-14-47-26_timeSeries.csv
    1693 data/imf/FDI_11-20-2018 21-39-31-89_timeSeries.csv
 1247714 data/imf/GFSR_11-25-2018 19-23-39-70_timeSeries.csv
   16732 data/imf/IRFCL_11-25-2018 19-13-18-05_timeSeries.csv
    7846 data/imf/ITS_11-14-2018 15-14-06-02_timeSeries.csv
    7425 data/imf/PPLT_11-25-2018 19-25-01-32_timeSeries.csv
 1661440 total


In [61]:
time_values = [str('%sM%s' % (y, m)) for m in list(range(1, 13)) for y in list(range(1960, 2018))]
imf_columns = ['Country Name', 'Indicator Name'] + time_values

In [62]:
imf_country_aggregates = ['Euro Area']

In [63]:
def load_imf_monthly(file_name, indicators, imf_columns, country_aggregates):
    csv_df = pd.read_csv('imf/%s' % file_name).fillna(0)
    base_df = csv_df.loc[csv_df['Attribute'] == 'Value'].drop(columns=['Attribute'])
    monthly_df = base_df.loc[(base_df['Indicator Name'].isin(indicators))]
    imf_df = monthly_df[imf_columns].fillna(0)
    df = pd.melt(imf_df, id_vars=['Country Name', 'Indicator Name'], var_name='date', value_name='value')
    df['date'] = pd.to_datetime(df['date'], format='%YM%m')
    df.columns = ['country', 'indicator', 'date', 'value']
    return df.loc[~df['country'].isin(country_aggregates)]

In [64]:
imf_pplt_df = load_imf_monthly('PPLT_11-25-2018 19-25-01-32_timeSeries.csv', indicator_list, imf_columns, imf_country_aggregates)

In [65]:
imf_cpi_df = load_imf_monthly('CPI_11-25-2018 19-14-47-26_timeSeries.csv', indicator_list, imf_columns, imf_country_aggregates)

In [66]:
imf_df = pd.concat([imf_cpi_df, imf_pplt_df], join='outer')

In [67]:
imf_df.size

0

In [68]:
imf_df.head(15)

,country,indicator,date,value


In [69]:
len(imf_df['country'].unique())

0

In [70]:
imf_countries = sorted(list(imf_df['country'].unique()))

### Load world bank yearly data

In [71]:
%%bash
wc -l world_bank/*.csv

   33534 data/world_bank/ECON.csv
    9589 data/world_bank/HNP.csv
      38 data/world_bank/HNP_indicator_definitions.csv
   36174 data/world_bank/POP.csv
   79335 total


In [72]:
wb_country_aggregates = ['nan', 'Lower middle income', 'Post-demographic dividend', 'High income',
                         'Pre-demographic dividend', 'East Asia & Pacific (IDA & IBRD countries)',
                         'Europe & Central Asia (excluding high income)', 'Heavily indebted poor countries (HIPC)',
                         'Caribbean small states', 'Pacific island small states', 'Middle income',
                         'Late-demographic dividend', 'OECD members', 'IDA & IBRD total', 'Not classified', 
                         'East Asia & Pacific (excluding high income)',
                         'Latin America & the Caribbean (IDA & IBRD countries)', 'Low income', 'Low & middle income',
                         'IDA blend', 'IBRD only', 'Sub-Saharan Africa (excluding high income)', 
                         'Fragile and conflict affected situations', 'Europe & Central Asia (IDA & IBRD countries)',
                         'Euro area', 'Other small states', 'Europe & Central Asia', 'Arab World',
                         'Latin America & Caribbean (excluding high income)', 
                         'Sub-Saharan Africa (IDA & IBRD countries)', 'Early-demographic dividend', 'IDA only',
                         'Small states', 'Middle East & North Africa (excluding high income)', 'East Asia & Pacific',
                         'South Asia', 'European Union', 'Least developed countries: UN classification',
                         'Middle East & North Africa (IDA & IBRD countries)', 'Upper middle income',
                         'South Asia (IDA & IBRD)', 'Central Europe and the Baltics', 'Sub-Saharan Africa', 
                         'Latin America & Caribbean', 'Middle East & North Africa', 'IDA total', 'North America',
                         'Last Updated: 11/14/2018', 'Data from database: World Development Indicators', 'World']

In [73]:
wb_cols = ['Country Name', 'Series Name'] + [str('%s [YR%s]' % (y, y)) for y in list(range(1960, 2018))]

In [74]:
def load_wb_yearly(file_name, indicators, wb_columns, country_aggregates):
    csv_df = pd.read_csv('world_bank/%s' % file_name).fillna(0)
    base_df = csv_df.loc[(csv_df['Series Name'].isin(indicators))]
    wb_df = base_df[wb_columns].fillna(0)
    df = pd.melt(wb_df, id_vars=['Country Name', 'Series Name'], var_name='date', value_name='value')
    df['date'] = pd.to_datetime(df['date'].map(lambda x: int(x.split(' ')[0])), format='%Y')
    df.columns = ['country', 'indicator', 'date', 'value']
    return df.loc[~df['country'].isin(country_aggregates)]

In [75]:
wb_econ_df = load_wb_yearly('ECON.csv', indicator_list, wb_cols, wb_country_aggregates)

In [76]:
wb_hnp_df = load_wb_yearly('HNP.csv', indicator_list, wb_cols, wb_country_aggregates)

In [77]:
wb_pop_df = load_wb_yearly('POP.csv', indicator_list, wb_cols, wb_country_aggregates)

In [78]:
wb_df = pd.concat([wb_econ_df, wb_hnp_df, wb_pop_df], join='outer')

In [79]:
wb_df.size

805504

In [80]:
wb_df.head(15)

,country,indicator,date,value
0,Afghanistan,"Energy imports, net (% of energy use)",1960-01-01,0.00
1,Afghanistan,Exports of goods and services (% of GDP),1960-01-01,4.13
2,Afghanistan,Exports of goods and services (constant 2010 US$),1960-01-01,0.00
3,Afghanistan,Food exports (% of merchandise exports),1960-01-01,0.00
4,Afghanistan,Food imports (% of merchandise imports),1960-01-01,0.00
5,Afghanistan,GNI per capita (constant 2010 US$),1960-01-01,0.00
6,Afghanistan,High-technology exports (% of manufactured exp...,1960-01-01,0.00
7,Afghanistan,Households and NPISHs Final consumption expend...,1960-01-01,0.00
8,Afghanistan,Imports of goods and services (% of GDP),1960-01-01,7.02
9,Afghanistan,Imports of goods and services (constant 2010 US$),1960-01-01,0.00


In [81]:
len(wb_df['country'].unique())

217

In [82]:
wb_countries = sorted(list(wb_df['country'].unique()))

### Combine the two datasets

In [83]:
imf_specific = [country for country in imf_countries if country not in wb_countries]

In [84]:
len(imf_specific)

0

In [85]:
imf_to_wb_country_map = {
    'Afghanistan, Islamic Republic of': 'Afghanistan',
    'Armenia, Republic of': 'Armenia',
    'Azerbaijan, Republic of': 'Azerbaijan',
    'Bahrain, Kingdom of': 'Bahrain',
    'China, P.R.: Hong Kong': 'Hong Kong SAR, China',
    'China, P.R.: Macao': 'Macao SAR, China',
    'China, P.R.: Mainland': 'China',
    'Congo, Democratic Republic of': 'Congo, Dem. Rep.',
    'Congo, Republic of': 'Congo, Rep.',
    'Egypt': 'Egypt, Arab Rep.',
    'French Territories: New Caledonia': 'New Caledonia',
    'Iran, Islamic Republic of': 'Iran',
    'Korea, Republic of': 'Korea, Rep.',
    'Kosovo, Republic of': 'Kosovo',
    "Lao People's Democratic Republic": 'Lao PDR',
    'Serbia, Republic of': 'Serbia',
    'Sint Maarten': 'Sint Maarten (Dutch part)',
    'Timor-Leste, Dem. Rep. of': 'Timor-Leste',
    'Venezuela, Republica Bolivariana de': 'Venezuela, RB',
    'Venezuela, República Bolivariana de': 'Venezuela, RB',
    'Yemen, Republic of': 'Yemen'
}

In [86]:
imf_df = imf_df.replace({'country': imf_to_wb_country_map})

In [87]:
trade_df = pd.concat([wb_df, imf_df], join='outer')

In [88]:
trade_df.size

805504

In [89]:
trade_df.head(15)

,country,indicator,date,value
0,Afghanistan,"Energy imports, net (% of energy use)",1960-01-01,0.00
1,Afghanistan,Exports of goods and services (% of GDP),1960-01-01,4.13
2,Afghanistan,Exports of goods and services (constant 2010 US$),1960-01-01,0.00
3,Afghanistan,Food exports (% of merchandise exports),1960-01-01,0.00
4,Afghanistan,Food imports (% of merchandise imports),1960-01-01,0.00
5,Afghanistan,GNI per capita (constant 2010 US$),1960-01-01,0.00
6,Afghanistan,High-technology exports (% of manufactured exp...,1960-01-01,0.00
7,Afghanistan,Households and NPISHs Final consumption expend...,1960-01-01,0.00
8,Afghanistan,Imports of goods and services (% of GDP),1960-01-01,7.02
9,Afghanistan,Imports of goods and services (constant 2010 US$),1960-01-01,0.00


In [90]:
indicators = sorted(list(trade_df['indicator'].unique()))

In [91]:
assert len(indicators) == len(indicator_list), 'The number of retrieved variables (%s) does not match the number of specified variables (%s).\nThe following variables are missing:\n\n %s' % (len(indicators), len(indicator_list), [i for i in indicator_list if i not in indicators])

In [92]:
trade_df.to_csv('model_one/trade.csv', sep=';', index=False)